#Import libraries needed for analysis

In [1]:
import pandas as pd
import numpy as np

# Data import and cleaning

In [13]:
df.train = pd.read_csv('train.csv', header=0)
df.bids = pd.read_csv('bids.csv', header=0)
df.test = pd.read_csv('test.csv', header=0)

# Explore data

Check how many examples do we have in the datasets provided by Facebook.

In [8]:
%%bash
wc -l *.csv

 7656335 bids.csv
    4701 sampleSubmission.csv
    4701 test.csv
    2014 train.csv
 7667751 total


## Bidder dataset

In [16]:
df.train.head(3)

,bidder_id,payment_account,address,outcome
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0


In [17]:
df.train.describe()

,outcome
count,2013.000000
mean,0.051167
std,0.220394
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Our training data consist of 2013 entries. Outcome is 1 when the bidder is a robot.

In [19]:
df.train[df.train['outcome']==1]['outcome'].count()

103

103 entries in the training dataset are robots.

In [20]:
df.train[df.train['outcome']==1].head(10)

,bidder_id,payment_account,address,outcome
21,5354c02817e47f28e60e44a40ca5e48dll4sz,1862ef3b7965993cc975a1161010e810w2bwo,58e5dc4d38d886f6c7abf0c264a51cb7alhj2,1
50,9434778d2268f1fa2a8ede48c0cd05c097zey,ee3295c7b31a1e9116ebf0cf2df68ed1ac8ts,627fe7481f01372c02333477471cb237qt2vf,1
51,aabc211b4cf4d29e4ac7e7e361371622pockb,a3d2de7675556553a5f08e4c88d2c228pxtr6,a3d2de7675556553a5f08e4c88d2c228bxn98,1
58,68d468e15e0de0bab9694cdca8e799eb2j8kn,a3d2de7675556553a5f08e4c88d2c228vf08k,767150cd5a97ed796bfc58456f5500250dsv7,1
77,1cbd9081de4481ada66f1853737f7684ddjd1,a3d2de7675556553a5f08e4c88d2c228buft1,a3d2de7675556553a5f08e4c88d2c228pijoz,1
86,96b90af2d3eea4d3b2bc725e1c92e212qu52g,39ef2afbc0cde265550cafb40eb3fd8erzlrm,0e6dfa874409a90bc0fbf0d531475241cvw1o,1
139,15972a431aed117840dfce5556242366c3fwq,4b5c8f52a678e51ebe0ee3fc26da66902e2l7,865e5987e0c8fc7f6301b5571d688d13pu5j1,1
148,efc61ad6db17265a4bac85f77b154ef414tpp,a3d2de7675556553a5f08e4c88d2c22850etl,a3d2de7675556553a5f08e4c88d2c228d6pnk,1
165,3d9910a238a0f282ad3132dd6fc981c8tv3tg,a3d2de7675556553a5f08e4c88d2c2281ehah,c856c4f51e949635cb567f7321bc0c6at1y78,1
199,1af4685cda2979a608c4b6ad83473774qgicn,e35a74340ac6d90fcb4fe9b84c6eaa2135wqx,ceda5914ce08b87ec934e614452442903j17m,1


## Bid dataset

In [22]:
df.bids.head(3)

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3


In [32]:
df.bids.head(20000).groupby('bidder_id').bidder_id.nunique()

bidder_id
0051aef3fdeacdadba664b9b3b07e04e4coc6    1
0061edfc5b07ff3d70d693883a38d370oy4fs    1
01067975436d123f717ee5aba0dd4bbfa0937    1
0113d101ec6aabd354adac645a1ec3e82ln88    1
01255c2c7c5578c186873422fc00fd7afwk8k    1
01349288df20199905e719f0ff7ee771nwryj    1
019cf2d366df756c092c91e26f406acdozha7    1
01cda526658455000913950f20cf31a2q6nsf    1
02518a364d87661eed1453e251933862yvmjj    1
02a28b9de21bcb744bc2a7dedc008f91pbcpc    1
0306d10ab30a2f5cda41ae7ed77b8d31oehvv    1
037d9f10da403d8d5f94b6e2957a3702f6x07    1
03e884d3e7e3356608187d2345e4693fyt29x    1
0430923026291636fff20fa1707fd34f0nayx    1
04bffaa5e353b8a37a66767e3d9b531din3rc    1
04c6b750b0552089c87ae03de72289cfcpfmh    1
04f53d51a9cac9391b1d35c109294c48apax1    1
051586c3297f5b992000048f3704550fxja6g    1
0547dbd3ecb4a293913f138d60b586fdf4ane    1
054b51d059fe981ca8cc4e0d4c49aabb5mck7    1
055282b75717345ba6f53d40e9e6d6c56qrzp    1
0565a780f2c5b46837bbe9739ef15ea92fdu0    1
0565d7d345ac29442b26cd9ffe2b3b841wwnb    1
0